In [85]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [86]:
file_path = '/home/carolus/Documents/school/green_ia/data/11_data/11_test_01.jsonl'
#file_path = '/home/carolus/Documents/school/green_ia/data/01_data/01_openfoodfacts_03.jsonl'
num_samples = 500

In [87]:
def count_objects_in_jsonl(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            count += 1
    return count
print(f"nombre objets {file_path}: {count_objects_in_jsonl(file_path)}")


nombre objets /home/carolus/Documents/school/green_ia/data/11_data/11_test_01.jsonl: 134616


In [88]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [89]:
df = sample_jsonl_file(file_path, num_samples)

In [90]:
nb_empty = (df['ecoscore_score'] == 49.5).sum() # remplacer la valeur par la médiane calculée !!!!!!!!!!!
nb_total = len(df)
pourcentage_empty = (nb_empty / nb_total) * 100
print(f"percetange empty in ecoscore_score, replaced by median : {pourcentage_empty:.2f}%")

percetange empty in ecoscore_score, replaced by median : 6.60%


In [91]:
df.tail(100)

,groups,ecoscore_tags,ecoscore_score,countries,categories,labels_note,text_data
400,0.0,4.0,15.0,60.0,"snacks, sweet-snacks, cocoa-and-its-products, ...",0.000000,empty caramel pointe de sel - chocolat noir empty
401,1.0,1.0,63.0,60.0,"condiments, sauces, salad-dressings, vinaigret...",0.000000,"plastique water, alcohol-vinegar, vinegar, ex..."
402,4.0,4.0,6.0,64.0,"meats-and-their-products, meat-preparations, c...",0.222222,empty hähnchenschnitzel empty
403,4.0,3.0,34.0,60.0,"dairies, fermented-foods, fermented-milk-produ...",0.000000,empty camembert empty
404,4.0,1.0,75.0,138.0,"plant-based-foods-and-beverages, plant-based-f...",0.000000,empty laurel leaves empty
405,0.0,2.0,50.0,60.0,"snacks, breakfasts, sweet-snacks, biscuits-and...",0.333333,"empty weet-bix go wholemeal-cereal, cereal, su..."
406,0.0,3.0,32.0,60.0,"snacks, sweet-snacks, confectioneries, candies",0.000000,"empty peanut chocolate candies, peanut milk-ch..."
407,1.0,4.0,19.0,139.0,"dairies, fats, spreads, spreadable-fats, anima...",0.000000,empty empty
408,0.0,3.0,30.0,60.0,"breakfasts, spreads, sweet-spreads, hazelnut-s...",0.222222,"empty nocciolata skimmed-milk-powder, dairy, m..."
409,6.0,1.0,75.0,60.0,"farming-products, eggs, chicken-eggs, barn-chi...",0.333333,cartone uova biologiche di arianna vulpiani empty


In [92]:
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"warning, column: '{column_name}'does not exists")
        return
    data = df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',  
    ))
    fig.update_layout(
        title=f'treemap column: {column_name}',
        paper_bgcolor='white',  
        plot_bgcolor='white',   
        font_color='black'      
    )
    fig.show()

In [93]:
colonnes_a_travailler = ['ecoscore_tags', 'ecoscore_score', 'countries', 'labels_note', 'groups']  
df = load_jsonl_to_dataframe(file_path)
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [94]:
file_path_csv = '/home/carolus/Documents/school/green_ia/scripts/validation_predictions.csv'
df_csv = pd.read_csv(file_path_csv)

FileNotFoundError: [Errno 2] No such file or directory: '/home/carolus/Documents/school/green_ia/scripts/validation_predictions.csv'

In [ ]:
df_csv.head(60)

In [ ]:
df_csv['difference'] = df_csv['true'] - df_csv['predictions']
max_diff = df_csv['difference'].max()
min_diff = df_csv['difference'].min()
mean_diff = df_csv['difference'].mean()  
median_diff = df_csv['difference'].median()  
total_rows = df_csv.shape[0]
df_csv['difference'] = df_csv['true'] - df_csv['predictions']
count_greater_than_2 = (df_csv['difference'].abs() > 2).sum()
percentage = (count_greater_than_2 / total_rows) * 100

print(f"Écart maximum: {max_diff}")
print(f"Écart minimum: {min_diff}")
print(f"Écart moyen: {mean_diff}")
print(f"Écart médian: {median_diff}")
print(f"poucentage lignes écart supérieur à 2% : {percentage}%")